<a href="https://colab.research.google.com/github/kihyuk-nam/kihyuk-nam/blob/master/Experiments_Claude_3_5_Sonnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
# Clinical reasoning test
---
- LLMs used: ChatGPT-4o
- Datasets:
  - Private:
    - [IDBR_2023_Question_Set_A (A set)](https://docs.google.com/spreadsheets/d/1ZvWyUuHco_2F3dFxDv0_g_3qkOYE2RXpRXk6IUoYyBg/edit?usp=drive_link)
    - [IDBR_2023_Question_Set_B (B set)](https://docs.google.com/spreadsheets/d/1NPBE2MaPc-ekcOBqyfK7hjJ_y9ecpjK-raY4o6WWHzE/edit?usp=drive_link)
  - Public:
    - MedQA
    - PubMedQA
- Prompt from [the paper](https://www.nature.com/articles/s41746-024-01010-1)


---
# Definitions
---

## 1. key information
- API key
- dataset url
- paths for the result files (.csv, .log)


In [ ]:
# Setting up OpenAI API key
from google.colab import userdata # get the key from the colab secrets
import os
os.environ['ANTHROPIC_API_KEY'] = userdata.get('ANTHROPIC_API_KEY_KIMNAM')
# openai.api_key = userdata.get('OPENAI_API_KEY') # deprecated

In [ ]:
# set info of input dataset file
url_case = 'https://docs.google.com/spreadsheets/d/1fdjvzc6ktNmD-fLFPl1Nk7ZL296ZFMJtcZQgr0UFOYQ/edit?usp=sharing' # IDBR_2023_Question_Set_A (B2-101 Cell)
#url = 'https://docs.google.com/spreadsheets/d/1NPBE2MaPc-ekcOBqyfK7hjJ_y9ecpjK-raY4o6WWHzE/edit?usp=sharing' # IDBR_2023_Question_Set_B
sheet_case = 'latest'
cell_col_case = 'B'
cell_row_case = '2'

# set info of prompt template file
url_prompt = 'https://docs.google.com/spreadsheets/d/12jZvwX0KDUm8C1VErSOTKx_J6wVWEF9mq-hqYrzE3ug/edit?usp=sharing' # Prompt Template (B2 Cell)
sheet_prompt = 'latest'
cell_col_prompt = 'B'
cell_row_promt = '2'

In [ ]:
# setting up the path for saving results (.log, .csv)
root_path = '/content/drive/MyDrive/2024/Clinical Reasoning/' # base directory
question_set = 'IDBR_2023_Question_Set_A' # 'IDBR_2023_Question_Set_B'
llm_type = 'Claude-3.5-Sonnet'
result_path = root_path + question_set + '_Result/' + llm_type + '/'

# check the path
print(result_path)
# should be one of the followings
#  '/content/drive/MyDrive/2024/Clinical Reasoning/IDBR_2023_Question_Set_A_Result/ChatGPT-4o/'
#  '/content/drive/MyDrive/2024/Clinical Reasoning/IDBR_2023_Question_Set_A_Result/Claude-3.5-Sonnet/'
#  '/content/drive/MyDrive/2024/Clinical Reasoning/IDBR_2023_Question_Set_B_Result/ChatGPT-4o/'
#  '/content/drive/MyDrive/2024/Clinical Reasoning/IDBR_2023_Question_Set_B_Result/Claude-3.5-Sonnet/'

/content/drive/MyDrive/2024/Clinical Reasoning/IDBR_2023_Question_Set_A_Result/Claude-3.5-Sonnet/


In [ ]:
# set the scope of the input (start ID ~ end ID)
start_row = 0
end_row = 99

---
# Execution
---

## 1. Setting up pre-requisites
- install dependencies
- mount into google drive
- set google drive paths for result (.log and .csv)

In [ ]:
# install dependencies
!pip install --upgrade anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 891.9/891.9 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00


In [ ]:
# connect to Gogole Drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# Set output file path
from datetime import datetime
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
file_path = f"{result_path}result_{current_time}.csv"
log_path = f"{result_path}result_{current_time}.log"
print(file_path)
print(log_path)

/content/drive/MyDrive/2024/Clinical Reasoning/IDBR_2023_Question_Set_A_Result/Claude-3.5-Sonnet/result_20240926_141533.csv
/content/drive/MyDrive/2024/Clinical Reasoning/IDBR_2023_Question_Set_A_Result/Claude-3.5-Sonnet/result_20240926_141533.log


In [ ]:
# move into the result_path
%cd $result_path
!ls

/content/drive/MyDrive/2024/Clinical Reasoning/IDBR_2023_Question_Set_A_Result/Claude-3.5-Sonnet
Experiment-0829  Experiment-0901  Experiment-0905  Experiment-0908-CoT	휴지통


## 2. Preparing for the input data
- extract prompt from the template sheet
  - Prompt from [the paper](https://www.nature.com/articles/s41746-024-01010-1)
  - use CoT for differential diagnosis and analytical reasoning *only*
- extract only questions from the original datasets
- save them into a Pandas DataFrame object
- used the code from [the medium article](https://medium.com/stackademic/how-to-read-data-from-google-sheets-in-google-colab-2370f54a7fff)

In [ ]:
# pre-requisites for accessing google sheets
from google.colab import auth
import gspread
from google.auth import default
import pandas as pd

In [ ]:
# authenticating to google
# [TODO] check for any duplication (cf. code for mounting the drive)
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
# read all values from a sheet
def load_column(url, sheet_name='latest', column='B', start='2'):

    # 1. Open the Google Sheets document
    sh = gc.open_by_url(url)

    # 2. Selecting the first sheet
    worksheet = sh.get_worksheet(0)

    # 3. Select the sheet_name if any. Default is the first sheet
    if sheet_name:
        worksheet = sh.worksheet(sheet_name)
    else:
        worksheet = sh.get_worksheet(0)

    # 3. Get all questions from the specified column starting from the specified row
    col_letter = column.upper()
    start_cell = f'{col_letter}{start}'
    end_cell = f'{col_letter}{worksheet.row_count}'
    cell_range = f'{start_cell}:{end_cell}'

    data = worksheet.get_values(cell_range)
    data = [item[0] for item in data if item]  # Flatten the list and remove empty cells

    # 5. Convert to DataFrame
    df = pd.DataFrame(data)

    return df

In [ ]:
#[TODO] read_string = load_column(url_prompt, sheet_prompt, 'B', '2')
example = ""
cot = f"""
Following is an example of the medical question, which is answered by Differential diagnosis chain-of-thought (CoT) and Analytic reasoning CoT.

“Shortly after undergoing a bipolar prosthesis for a displaced femoral neck fracture of the left hip acquired after a fall the day before, an 80-year-old woman suddenly develops dyspnea. The surgery under general anesthesia with sevoflurane was uneventful, lasting 98 min, during which the patient maintained oxygen saturation readings of 100% on 8 l of oxygen. She has a history of hypertension, osteoporosis, and osteoarthritis of her right knee. Her medications include ramipril, naproxen, ranitidine, and a multivitamin. She appears cyanotic, drowsy, and is oriented only to person. Her temperature is 38.6 °C (101.5 °F), pulse is 135/min, respirations are 36/min, and blood pressure is 155/95 mm Hg. Pulse oximetry on room air shows an oxygen saturation of 81%. There are several scattered petechiae on the anterior chest wall. Laboratory studies show a hemoglobin concentration of 10.5 g/dl, a leukocyte count of 9000/mm3, a platelet count of 145,000/mm3, and a creatine kinase of 190 U/l. An ECG shows sinus tachycardia. What is the most likely diagnosis?

Differential diagnosis chain-of-thought (CoT): This patient has shortness of breath after a long bone surgery. The differential for this patient is pulmonary embolism, fat embolism, myocardial infarction, blood loss, anaphylaxis, or a drug reaction. The patient has petechiae which makes fat embolism more likely. This patient most likely has a fat embolism.

Analytic reasoning CoT: The patient recently had large bone surgery making fat emboli a potential cause because the bone marrow was manipulated. Petechiae can form in response to capillary inflammation caused by fat emboli. Fat micro globules cause CNS microcirculation occlusion causing confusion and altered mental status. Fat obstruction in the pulmonary arteries can cause tachycardia and shortness of breath as seen in this patient. This patient most likely has a fat embolism.”

Following the example, read the presentation of a medical question and provide the single best answer using the following two steps.
First, develop an initial preliminary answer by applying both the differential diagnosis chain of thought (CoT) and analytic reasoning CoT.
Then, reassess the relevant details from the case presentation and reevaluate the initial answer.
During the reevaluation process, if conflicting factors arise, such as drug interactions, clinical risks, or benefit-risk considerations, select the answer that minimizes potential harm to the patient.
After completing the reevaluation, provide both the initial preliminary answer and the final answer, with an explanation supported by the differential diagnosis CoT and analytic reasoning CoT.

Question:
"""
prompt_template = lambda comment: f'''\n{example} \n{question} \n'''

In [ ]:
# load the question column into 'questions'
cases = load_column(url_case)
#print(cases)

In [ ]:
# check the result (verbose)
for index, row in cases.iterrows():
  if 13 > index or index > 15:
    continue
  question = row[0]
  prompt = prompt_template(question)
  print(f"\n{index}:\n{prompt}\n")
    #for column in df.columns: # in case of multiple columns
        #value = row[column]
        #print(f"Row {index}, Column {column}: {value}")


13:

 
A 55-year-old woman from rural Connecticut seeks your advice on what to do about a tiny tick she removed from her abdomen five days previously. She does not know how long the tick had been there because she had been on a camping trip and not bathing regularly. She takes prednisone and methotrexate for her rheumatoid arthritis. She feels well and has no skin lesions. 

Assuming that the tick was likely Ixodes scapularis, you would: 
> Prescribe a single 200 mg dose of doxycycline 
> Prescribe a 10-day course of doxycycline, 100 mg BID 
> Prescribe a single 500 mg dose of amoxicillin 
> Prescribe a 10-day course of amoxicillin 500 mg TID 
> Prescribe nothing unless compatible manifestations develop 



14:

 
A 19-year-old male with no significant past medical history was found difficult to arouse by his roommate in his college dormitory room and was taken to the emergency department. There, he was found to have bacteremic meningococcal meningitis. The patient was started on ceft

## 3. Running the experiment
- running states are recorded into the .log file

In [ ]:
import anthropic # Don't forget to !pip install anthropic
import time # for delaying between api calls

# set the LLM to be used
client = anthropic.Anthropic()

results = [] # results to be generated by LLM
with open(log_path, 'w', encoding='utf-8', newline='') as f:
  for index, row in cases.iterrows():
    if start_row > index or index > end_row:
      continue
    question = row[0]
    prompt = prompt_template(question)
    print(f"processing #{index}")
    f.write(f"\n{index}:\n{prompt}\n")

    message=[{"role": "user", "content": prompt}]
    try:
      response = client.messages.create(
          model="claude-3-5-sonnet-20240620",
          messages=message,
          temperature=0,
          max_tokens=1000,
      )

      # extract text from 'response' and add it to 'result'
      # and add it to the results
      response_text = response.content[0].text
      results.append({'question': question, 'response': response_text})
      f.write(f"Response:\n{response_text}\n")
    except Exception as e:
      print(f"Error occurred: {e}")
      results.append({'question': question, 'response': f"Error: {str(e)}"})

  time.sleep(15)

df_results = pd.DataFrame(results)

processing #0
processing #1
processing #2
processing #3
processing #4
processing #5
processing #6
processing #7
processing #8
processing #9
processing #10
processing #11
processing #12
processing #13
processing #14
processing #15
processing #16
processing #17
processing #18
processing #19
processing #20
processing #21
processing #22
processing #23
processing #24
processing #25
processing #26
processing #27
processing #28
processing #29
processing #30
processing #31
processing #32
processing #33
processing #34
processing #35
processing #36
processing #37
processing #38
processing #39
processing #40
processing #41
processing #42
processing #43
processing #44
processing #45
processing #46
processing #47
processing #48
processing #49
processing #50
processing #51
processing #52
processing #53
processing #54
processing #55
processing #56
processing #57
processing #58
processing #59
processing #60
processing #61
processing #62
processing #63
processing #64
processing #65
processing #66
proce

In [ ]:
# check the result (concise)
print(df_results)

                                             question  \
0   A 59-year-old male is being treated for MSSA s...   
1   A 20-year-old woman presents with progressive ...   
2   In your community, there is an outbreak of cas...   
3   At a nuclear plant in Argentina there is an ou...   
4   A 40-year-old businessman develops diarrhea wh...   
..                                                ...   
95  Four adults became ill after having lunch toge...   
96  A 57-year-old man seeks attention for intermit...   
97  You are called by a 41-year-old friend who two...   
98  A 33-year-old bank executive is seen for inter...   
99  A 49-year-old oncologist noted an enlarged lym...   

                                             response  
0   Based on the information provided, the best ma...  
1   Based on the information provided, the most ap...  
2   Based on the information provided, the organis...  
3   Based on the information provided, the most li...  
4   Based on the information provid

## 4. Saving the result

In [ ]:
with open(file_path, 'w', encoding='utf-8', newline='') as f:
  df_results.to_csv(f, header=True, index=False)
  print("Results saved to results.csv")

Results saved to results.csv
